This Code give the output of apples with count and  locations within coordinate

In [ ]:
#这串代码实现将图上，坐标都去掉
import cv2
import numpy as np
import matplotlib.pyplot as plt

# 读取图片
def ReadImg(image_path):
    img = cv2.imread(image_path, 1)
    return img

# 高斯滤波
def GausBlur(src):
    dst = cv2.GaussianBlur(src, (5, 5), 1.5)
    return dst

# 开运算（替代 open_mor）
def open_mor(img):
    kernel = np.ones((5, 5), np.uint8)
    result = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
    return result

# 红色对象检测
def detect_red_objects(src):
    # 设定红色阈值范围
    lower_red = np.array([0, 0, 100])
    upper_red = np.array([100, 100, 255])

    # 根据阈值构建掩模
    mask = cv2.inRange(src, lower_red, upper_red)

    # 使用掩模提取红色对象
    red_objects = cv2.bitwise_and(src, src, mask=mask)

    return red_objects

# 轮廓拟合
def draw_shape(open_img, src):
    contours, hierarchy = cv2.findContours(open_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    apple_count = 0
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > 50:  # 假设红色对象的最小面积为100
            # 获取圆心和半径
            (x, y), radius = cv2.minEnclosingCircle(cnt)
            center = (int(x), int(y))
            radius = int(radius)

            # 绘制圆形
            cv2.circle(src, center, radius, (0, 0, 255), 2)

            # 绘制坐标，按照您的指示处理y坐标
            inverted_y = 185 - int(y)  # 修改这里
            cv2.putText(src, f'({center[0]}, {inverted_y})', (center[0] - 30, center[1] + 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 255, 255), 1, cv2.LINE_AA)
            apple_count += 1
            # 打印坐标
            print('苹果坐标:', x, inverted_y)
    return src, apple_count

# Watershed 分割
def watershed_segmentation(open_img, src):
    # 使用距离变换和阈值分割进行分水岭算法
    dist_transform = cv2.distanceTransform(open_img, cv2.DIST_L2, 5)
    _, sure_fg = cv2.threshold(dist_transform, 0.7 * dist_transform.max(), 255, 0)
    sure_fg = np.uint8(sure_fg)

    # 找到未知区域
    sure_bg = cv2.dilate(open_img, None, iterations=3)
    sure_bg = 255 - sure_bg

    # 标记未知区域
    _, markers = cv2.connectedComponents(sure_fg)
    markers = markers + 1
    markers[open_img == 0] = 0

    # 应用分水岭算法
    cv2.watershed(src, markers)


    return src

# Processing loop for all images
for i in range(1, 201):  # Assuming image numbers range from 1 to 200
    image_path = f'Attachment_2/{i}_processed.jpg'
    src = ReadImg(image_path)
    gaus_img = GausBlur(src)
    red_objects_img = detect_red_objects(gaus_img)
    gray_img = cv2.cvtColor(red_objects_img, cv2.COLOR_BGR2GRAY)
    _, thres_img = cv2.threshold(gray_img, 1, 255, cv2.THRESH_BINARY)
    open_img = open_mor(thres_img)
    result_img, apple_count = draw_shape(open_img, src)

    # Watershed segmentation
    result_img = watershed_segmentation(open_img, result_img)

    # Flip image vertically (doesn't affect Y-axis)
    flipped_img = np.flipud(result_img)

    # Output folder creation
    output_folder = 'abcdefg'
    os.makedirs(output_folder, exist_ok=True)

    # Create and save images without coordinate display and axes
    plt.imshow(cv2.cvtColor(flipped_img, cv2.COLOR_BGR2RGB))
    plt.axis('off')  # Turn off axis display
    output_path = os.path.join(output_folder, f'output_image_{i}.png')
    plt.savefig(output_path, bbox_inches='tight', pad_inches=0)  # Save image without padding
    plt.clf()  # Clear current plot for the next use
    print(f'Saved image to: {output_path}')

    # Display the processed image without coordinates
    plt.imshow(cv2.cvtColor(flipped_img, cv2.COLOR_BGR2RGB))
    plt.axis('off')  # Turn off axis display
    plt.show()

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

# 读取图片
def ReadImg(image_path):
    img = cv2.imread(image_path, 1)
    return img

# 高斯滤波
def GausBlur(src):
    dst = cv2.GaussianBlur(src, (5, 5), 1.5)
    return dst

# 开运算（替代 open_mor）
def open_mor(img):
    kernel = np.ones((5, 5), np.uint8)
    result = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
    return result

# 红色对象检测
def detect_red_objects(src):
    # 设定红色阈值范围
    lower_red = np.array([0, 0, 100])
    upper_red = np.array([100, 100, 255])

    # 根据阈值构建掩模
    mask = cv2.inRange(src, lower_red, upper_red)

    # 使用掩模提取红色对象
    red_objects = cv2.bitwise_and(src, src, mask=mask)

    return red_objects

# 轮廓拟合
def draw_shape(open_img, src):
    contours, hierarchy = cv2.findContours(open_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    apple_count = 0
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > 50:  # 假设红色对象的最小面积为100
            # 获取圆心和半径
            (x, y), radius = cv2.minEnclosingCircle(cnt)
            center = (int(x), int(y))
            radius = int(radius)

            # 描绘绿色的边框线
            cv2.drawContours(src, [cnt], 0, (0, 255, 0), 2)

            # 绘制圆形
            cv2.circle(src, center, radius, (0, 0, 255), 2)

            # 绘制坐标，按照您的指示处理y坐标
            inverted_y = 185 - int(y)  # 修改这里
            cv2.putText(src, f'({center[0]}, {inverted_y})', (center[0] - 30, center[1] + 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 255, 255), 1, cv2.LINE_AA)
            apple_count += 1
            # 打印坐标
            print('苹果坐标:', x, inverted_y)
    return src, apple_count

# Watershed 分割
def watershed_segmentation(open_img, src):
    # 使用距离变换和阈值分割进行分水岭算法
    dist_transform = cv2.distanceTransform(open_img, cv2.DIST_L2, 5)
    _, sure_fg = cv2.threshold(dist_transform, 0.7 * dist_transform.max(), 255, 0)
    sure_fg = np.uint8(sure_fg)

    # 找到未知区域
    sure_bg = cv2.dilate(open_img, None, iterations=3)
    sure_bg = 255 - sure_bg

    # 标记未知区域
    _, markers = cv2.connectedComponents(sure_fg)
    markers = markers + 1
    markers[open_img == 0] = 0

    # 应用分水岭算法
    cv2.watershed(src, markers)

    return src

# 处理所有图像
for i in range(1, 201):  # 假设图像编号从1到200
    image_path = f'Attachment_2/{i}_processed.jpg'
    src = ReadImg(image_path)
    gaus_img = GausBlur(src)
    red_objects_img = detect_red_objects(gaus_img)
    gray_img = cv2.cvtColor(red_objects_img, cv2.COLOR_BGR2GRAY)
    _, thres_img = cv2.threshold(gray_img, 1, 255, cv2.THRESH_BINARY)
    open_img = open_mor(thres_img)
    result_img, apple_count = draw_shape(open_img, src)

    # Watershed 分割
    result_img = watershed_segmentation(open_img, result_img)

    # 上下翻转图像，不影响Y轴
    flipped_img = np.flipud(result_img)

    # 处理所有图像
    output_folder = 'abcde'
    os.makedirs(output_folder, exist_ok=True)
    # 创建带有坐标轴的图像
    plt.imshow(cv2.cvtColor(flipped_img, cv2.COLOR_BGR2RGB))
    plt.gca().invert_yaxis()  # 反转Y轴
    plt.title(f'Processed Image {i}, Red Objects Count: {apple_count}')

    # 将带有坐标轴的图像保存到文件
    output_path = os.path.join(output_folder, f'output_image_{i}.png')
    plt.savefig(output_path)
    plt.clf()  # 清除当前图像，以便下一次使用

    print(f'保存图像到: {output_path}')

    # 显示翻转后的图像
    plt.imshow(cv2.cvtColor(flipped_img, cv2.COLOR_BGR2RGB))
    plt.gca().invert_yaxis()  # 反转Y轴
    plt.title(f'Processed Image {i}, Red Objects Count: {apple_count}')
    plt.show()

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

# 读取图片
def ReadImg(image_path):
    img = cv2.imread(image_path, 1)
    return img

# 高斯滤波
def GausBlur(src):
    dst = cv2.GaussianBlur(src, (5, 5), 1.5)
    return dst

# 开运算（替代 open_mor）
def open_mor(img):
    kernel = np.ones((5, 5), np.uint8)
    result = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
    return result

# 红色对象检测
def detect_red_objects(src):
    # 设定红色阈值范围
    lower_red = np.array([0, 0, 100])
    upper_red = np.array([100, 100, 255])

    # 根据阈值构建掩模
    mask = cv2.inRange(src, lower_red, upper_red)

    # 使用掩模提取红色对象
    red_objects = cv2.bitwise_and(src, src, mask=mask)

    return red_objects

# 轮廓拟合
def draw_shape(open_img, src):
    contours, hierarchy = cv2.findContours(open_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    apple_count = 0
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > 50:  # 假设红色对象的最小面积为100
            # 获取圆心和半径
            (x, y), radius = cv2.minEnclosingCircle(cnt)
            center = (int(x), int(y))
            radius = int(radius)

            # 描绘绿色的边框线
            cv2.drawContours(src, [cnt], 0, (0, 255, 0), 2)

            # 绘制圆形
            cv2.circle(src, center, radius, (0, 0, 255), 2)

            # 绘制坐标，按照您的指示处理y坐标
            inverted_y = 185 - int(y)  # 修改这里
            cv2.putText(src, f'({center[0]}, {inverted_y})', (center[0] - 30, center[1] + 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 255, 255), 1, cv2.LINE_AA)
            apple_count += 1
            # 打印坐标
            print('苹果坐标:', x, inverted_y)
    return src, apple_count

# Watershed 分割
def watershed_segmentation(open_img, src):
    # 使用距离变换和阈值分割进行分水岭算法
    dist_transform = cv2.distanceTransform(open_img, cv2.DIST_L2, 5)
    _, sure_fg = cv2.threshold(dist_transform, 0.7 * dist_transform.max(), 255, 0)
    sure_fg = np.uint8(sure_fg)

    # 找到未知区域
    sure_bg = cv2.dilate(open_img, None, iterations=3)
    sure_bg = 255 - sure_bg

    # 标记未知区域
    _, markers = cv2.connectedComponents(sure_fg)
    markers = markers + 1
    markers[open_img == 0] = 0

    # 应用分水岭算法
    cv2.watershed(src, markers)

    return src

# 处理所有图像
for i in range(1, 201):  # 假设图像编号从1到200
    image_path = f'Attachment_2/{i}_processed.jpg'
    src = ReadImg(image_path)
    gaus_img = GausBlur(src)
    red_objects_img = detect_red_objects(gaus_img)
    gray_img = cv2.cvtColor(red_objects_img, cv2.COLOR_BGR2GRAY)
    _, thres_img = cv2.threshold(gray_img, 1, 255, cv2.THRESH_BINARY)
    open_img = open_mor(thres_img)
    result_img, apple_count = draw_shape(open_img, src)

    # Watershed 分割
    result_img = watershed_segmentation(open_img, result_img)

    # 上下翻转图像，不影响Y轴
    flipped_img = np.flipud(result_img)

    # 处理所有图像
    output_folder = 'abcde'
    os.makedirs(output_folder, exist_ok=True)
    # 创建带有坐标轴的图像
    plt.imshow(cv2.cvtColor(flipped_img, cv2.COLOR_BGR2RGB))
    plt.gca().invert_yaxis()  # 反转Y轴
    plt.title(f'Processed Image {i}, Red Objects Count: {apple_count}')

    # 将带有坐标轴的图像保存到文件
    output_path = os.path.join(output_folder, f'output_image_{i}.png')
    plt.savefig(output_path)
    plt.clf()  # 清除当前图像，以便下一次使用

    print(f'保存图像到: {output_path}')

    # 显示翻转后的图像
    plt.imshow(cv2.cvtColor(flipped_img, cv2.COLOR_BGR2RGB))
    plt.gca().invert_yaxis()  # 反转Y轴
    plt.title(f'Processed Image {i}, Red Objects Count: {apple_count}')
    plt.show()

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

# 读取图片
def ReadImg(image_path):
    img = cv2.imread(image_path, 1)
    return img

# 高斯滤波
def GausBlur(src):
    dst = cv2.GaussianBlur(src, (5, 5), 1.5)
    return dst

# 开运算（替代 open_mor）
def open_mor(img):
    kernel = np.ones((5, 5), np.uint8)
    result = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
    return result

# 红色对象检测
def detect_red_objects(src):
    # 设定红色阈值范围
    lower_red = np.array([0, 0, 100])
    upper_red = np.array([100, 100, 255])

    # 根据阈值构建掩模
    mask = cv2.inRange(src, lower_red, upper_red)

    # 使用掩模提取红色对象
    red_objects = cv2.bitwise_and(src, src, mask=mask)

    return red_objects

# 轮廓拟合
def draw_shape(open_img, src):
    contours, hierarchy = cv2.findContours(open_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    apple_count = 0
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > 50:  # 假设红色对象的最小面积为100
            # 获取圆心和半径
            (x, y), radius = cv2.minEnclosingCircle(cnt)
            center = (int(x), int(y))
            radius = int(radius)

            # 描绘绿色的边框线
            cv2.drawContours(src, [cnt], 0, (0, 255, 0), 2)

            # 绘制圆形
            cv2.circle(src, center, radius, (0, 0, 255), 2)

            apple_count += 1

    return src, apple_count

# Watershed 分割
def watershed_segmentation(open_img, src):
    # 使用距离变换和阈值分割进行分水岭算法
    dist_transform = cv2.distanceTransform(open_img, cv2.DIST_L2, 5)
    _, sure_fg = cv2.threshold(dist_transform, 0.7 * dist_transform.max(), 255, 0)
    sure_fg = np.uint8(sure_fg)

    # 找到未知区域
    sure_bg = cv2.dilate(open_img, None, iterations=3)
    sure_bg = 255 - sure_bg

    # 标记未知区域
    _, markers = cv2.connectedComponents(sure_fg)
    markers = markers + 1
    markers[open_img == 0] = 0

    # 应用分水岭算法
    cv2.watershed(src, markers)

    return src

# 处理所有图像
for i in range(1, 201):  # 假设图像编号从1到200
    image_path = f'Attachment_2/{i}_processed.jpg'
    src = ReadImg(image_path)
    gaus_img = GausBlur(src)
    red_objects_img = detect_red_objects(gaus_img)
    gray_img = cv2.cvtColor(red_objects_img, cv2.COLOR_BGR2GRAY)
    _, thres_img = cv2.threshold(gray_img, 1, 255, cv2.THRESH_BINARY)
    open_img = open_mor(thres_img)
    result_img, apple_count = draw_shape(open_img, src)

    # Watershed 分割
    result_img = watershed_segmentation(open_img, result_img)

    # 上下翻转图像，不影响Y轴
    flipped_img = np.flipud(result_img)

    # 处理所有图像
    output_folder = 'abcde'
    os.makedirs(output_folder, exist_ok=True)
    # 创建带有坐标轴的图像
    plt.imshow(cv2.cvtColor(flipped_img, cv2.COLOR_BGR2RGB))
    plt.gca().invert_yaxis()  # 反转Y轴
    plt.title(f'Processed Image {i}, Red Objects Count: {apple_count}')

    # 将带有坐标轴的图像保存到文件
    output_path = os.path.join(output_folder, f'output_image_{i}.png')
    plt.savefig(output_path)
    plt.clf()  # 清除当前图像，以便下一次使用

    print(f'保存图像到: {output_path}')

    # 显示翻转后的图像
    plt.imshow(cv2.cvtColor(flipped_img, cv2.COLOR_BGR2RGB))
    plt.gca().invert_yaxis()  # 反转Y轴
    plt.title(f'Processed Image {i}, Red Objects Count: {apple_count}')
    plt.show()

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

# 读取图片
def ReadImg(image_path):
    img = cv2.imread(image_path, 1)
    return img

# 高斯滤波
def GausBlur(src):
    dst = cv2.GaussianBlur(src, (5, 5), 1.5)
    return dst

# 开运算（替代 open_mor）
def open_mor(img):
    kernel = np.ones((5, 5), np.uint8)
    result = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
    return result

# 红色对象检测
def detect_red_objects(src):
    # 设定红色阈值范围
    lower_red = np.array([0, 0, 100])
    upper_red = np.array([100, 100, 255])

    # 根据阈值构建掩模
    mask = cv2.inRange(src, lower_red, upper_red)

    # 使用掩模提取红色对象
    red_objects = cv2.bitwise_and(src, src, mask=mask)

    return red_objects

# 轮廓拟合
def draw_shape(open_img, src):
    contours, hierarchy = cv2.findContours(open_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    apple_count = 0
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > 50:  # 假设红色对象的最小面积为100
            # 获取圆心和半径
            (x, y), radius = cv2.minEnclosingCircle(cnt)
            center = (int(x), int(y))
            radius = int(radius)

            # 描绘绿色的边框线
            cv2.drawContours(src, [cnt], 0, (0, 255, 0), 2)

            # 绘制圆形
            cv2.circle(src, center, radius, (0, 0, 255), 2)

            apple_count += 1

    return src, apple_count

# Watershed 分割
def watershed_segmentation(open_img, src):
    # 使用距离变换和阈值分割进行分水岭算法
    dist_transform = cv2.distanceTransform(open_img, cv2.DIST_L2, 5)
    _, sure_fg = cv2.threshold(dist_transform, 0.7 * dist_transform.max(), 255, 0)
    sure_fg = np.uint8(sure_fg)

    # 找到未知区域
    sure_bg = cv2.dilate(open_img, None, iterations=3)
    sure_bg = 255 - sure_bg

    # 标记未知区域
    _, markers = cv2.connectedComponents(sure_fg)
    markers = markers + 1
    markers[open_img == 0] = 0

    # 应用分水岭算法
    cv2.watershed(src, markers)

    return src

# 处理所有图像
for i in range(1, 201):  # 假设图像编号从1到200
    image_path = f'Attachment_2/{i}_processed.jpg'
    src = ReadImg(image_path)
    gaus_img = GausBlur(src)
    red_objects_img = detect_red_objects(gaus_img)
    gray_img = cv2.cvtColor(red_objects_img, cv2.COLOR_BGR2GRAY)
    _, thres_img = cv2.threshold(gray_img, 1, 255, cv2.THRESH_BINARY)
    open_img = open_mor(thres_img)
    result_img, apple_count = draw_shape(open_img, src)

    # Watershed 分割
    result_img = watershed_segmentation(open_img, result_img)

    # 上下翻转图像，不影响Y轴
    flipped_img = np.flipud(result_img)

    # 处理所有图像
    output_folder = 'abcde'
    os.makedirs(output_folder, exist_ok=True)
    # 创建带有坐标轴的图像
    plt.imshow(cv2.cvtColor(flipped_img, cv2.COLOR_BGR2RGB))
    plt.gca().invert_yaxis()  # 反转Y轴
    plt.xticks([])  # 移除X轴坐标标签
    plt.yticks([])  # 移除Y轴坐标标签
    plt.title(f'Processed Image {i}, Red Objects Count: {apple_count}')

    # 将带有坐标轴的图像保存到文件
    output_path = os.path.join(output_folder, f'output_image_{i}.png')
    plt.savefig(output_path)
    plt.clf()  # 清除当前图像，以便下一次使用

    print(f'保存图像到: {output_path}')

    # 显示翻转后的图像
    plt.imshow(cv2.cvtColor(flipped_img, cv2.COLOR_BGR2RGB))
    plt.gca().invert_yaxis()  # 反转Y轴
    plt.xticks([])  # 移除X轴坐标标签
    plt.yticks([])  # 移除Y轴坐标标签
    plt.title(f'Processed Image {i}, Red Objects Count: {apple_count}')
    plt.show()